
# 2.1
Is momentum still profitable? (a) Fill Table 1 for `r_mom:FF` across the full sample and the requested subsamples. (b) Use the same cuts on the loser and winner legs to see whether momentum changed materially over time. (c) Discuss whether the evidence continues to support AQR's claim that momentum is an essential sleeve, even if expected returns are compressed after implementation costs.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display

plt.style.use("seaborn-v0_8")

DATA_PATH = Path("momentum_data.xlsx")
END_DATE = "2024-12-31"
ANN_FACTOR = 12

def load_momentum_panel(path: Path) -> pd.DataFrame:
    xls = pd.ExcelFile(path)

    def prep(name):
        df = pd.read_excel(xls, sheet_name=name)
        df["Date"] = pd.to_datetime(df["Date"])
        return df.set_index("Date").sort_index()

    factors = prep("factors (excess returns)").rename(columns={"MKT": "r_m", "HML": "r_v", "SMB": "r_s"})
    momentum = prep("momentum (excess returns)").rename(columns={"UMD": "r_mom"})
    deciles = prep("deciles (total returns)").rename(columns={
        "Lo PRIOR": "r_mom1",
        "PRIOR 2": "r_mom2",
        "PRIOR 3": "r_mom3",
        "PRIOR 4": "r_mom4",
        "PRIOR 5": "r_mom5",
        "PRIOR 6": "r_mom6",
        "PRIOR 7": "r_mom7",
        "PRIOR 8": "r_mom8",
        "PRIOR 9": "r_mom9",
        "Hi PRIOR": "r_mom10",
    })
    size = prep("size_sorts (total returns)").rename(columns={
        "SMALL HiPRIOR": "r_momSU",
        "SMALL LoPRIOR": "r_momSD",
        "BIG HiPRIOR": "r_momBU",
        "BIG LoPRIOR": "r_momBD",
    })[["r_momSU", "r_momSD", "r_momBU", "r_momBD"]]
    rf = prep("risk-free rate").rename(columns={"RF": "r_f"})

    idx = factors.index
    for frame in (momentum, deciles, size, rf):
        idx = idx.intersection(frame.index)
    idx = idx[(idx >= "1927-01-31") & (idx <= END_DATE)]

    factors, momentum, deciles, size, rf = (frame.reindex(idx) for frame in (factors, momentum, deciles, size, rf))
    deciles_ex = deciles.sub(rf["r_f"], axis=0)
    size_ex = size.sub(rf["r_f"], axis=0)

    panel = pd.concat(
        [factors[["r_m", "r_v"]], momentum, deciles_ex, size_ex, rf],
        axis=1,
    ).dropna()

    panel["r_mom_ff"] = 0.5 * (panel["r_momBU"] + panel["r_momSU"]) - 0.5 * (panel["r_momBD"] + panel["r_momSD"])
    panel["r_momU"] = panel[["r_mom8", "r_mom9", "r_mom10"]].mean(axis=1)
    panel["r_momU_ff"] = 0.5 * (panel["r_momBU"] + panel["r_momSU"])
    panel["r_momD1"] = panel["r_mom10"] - panel["r_mom1"]
    panel["r_momD3"] = panel[["r_mom8", "r_mom9", "r_mom10"]].mean(axis=1) - panel[["r_mom1", "r_mom2", "r_mom3"]].mean(axis=1)
    panel["r_momD5"] = panel[["r_mom6", "r_mom7", "r_mom8", "r_mom9", "r_mom10"]].mean(axis=1) - panel[["r_mom1", "r_mom2", "r_mom3", "r_mom4", "r_mom5"]].mean(axis=1)
    panel["r_momS"] = panel["r_momSU"] - panel["r_momSD"]
    panel["r_momB"] = panel["r_momBU"] - panel["r_momBD"]

    return panel

monthly = load_momentum_panel(DATA_PATH)

sample_windows = {
    "Full (1927-2024)": ("1927-01-31", "2024-12-31"),
    "Pre-1994 (1927-1993)": ("1927-01-31", "1993-12-31"),
    "1994-2008": ("1994-01-31", "2008-12-31"),
    "2009-2024": ("2009-01-31", "2024-12-31"),
}

series_labels = {
    "r_mom_ff": "r_mom:FF",
    "r_mom": "Generic momentum (UMD)",
    "r_mom1": "Loser decile r_mom(1)",
    "r_mom10": "Winner decile r_mom(10)",
    "r_momSU": "Small winners r_momSU",
    "r_momBD": "Big losers r_momBD",
    "r_momU": "Long-only momentum (deciles)",
    "r_momU_ff": "Long-only momentum (r_momU:FF)",
    "r_momD1": "1-decile long-short",
    "r_momD3": "3-decile long-short",
    "r_momD5": "5-decile long-short",
    "r_momS": "Small-stock momentum",
    "r_momB": "Big-stock momentum",
}

metric_order = ["Mean % (ann)", "Vol % (ann)", "Sharpe", "Skew", "Corr r_m", "Corr r_v"]
metric_rename = {
    "Mean % (ann)": "Mean (% p.a.)",
    "Vol % (ann)": "Vol (% p.a.)",
    "Sharpe": "Sharpe",
    "Skew": "Skew",
    "Corr r_m": "Corr vs r_m",
    "Corr r_v": "Corr vs r_v",
}

def summarize_series(window: pd.DataFrame, column: str) -> dict:
    subset = window[[column, "r_m", "r_v"]].dropna()
    if subset.empty:
        return {}
    series = subset[column]
    mean_m = series.mean()
    vol_m = series.std(ddof=1)
    sharpe = np.nan if vol_m == 0 else (mean_m / vol_m) * np.sqrt(ANN_FACTOR)
    return {
        "Mean % (ann)": mean_m * ANN_FACTOR * 100,
        "Vol % (ann)": vol_m * np.sqrt(ANN_FACTOR) * 100,
        "Sharpe": sharpe,
        "Skew": series.skew(),
        "Corr r_m": series.corr(subset["r_m"]),
        "Corr r_v": series.corr(subset["r_v"]),
    }

def build_window_summary(columns, windows):
    rows = []
    for col in columns:
        for label, (start, end) in windows.items():
            window = monthly.loc[(monthly.index >= start) & (monthly.index <= end)]
            stats = summarize_series(window, col)
            if not stats:
                continue
            stats.update({"Series": col, "Period": label})
            rows.append(stats)
    df = pd.DataFrame(rows)
    return df.set_index(["Series", "Period"]).sort_index()

def compute_period_stats(series_names, start, end, include_value_corr=True):
    window = monthly.loc[(monthly.index >= start) & (monthly.index <= end)]
    rows = []
    for col in series_names:
        subset = window[[col, "r_m", "r_v"]].dropna()
        if subset.empty:
            continue
        series = subset[col]
        mean_m = series.mean()
        vol_m = series.std(ddof=1)
        row = {
            "Series": col,
            "Mean % (ann)": mean_m * ANN_FACTOR * 100,
            "Vol % (ann)": vol_m * np.sqrt(ANN_FACTOR) * 100,
            "Sharpe": np.nan if vol_m == 0 else (mean_m / vol_m) * np.sqrt(ANN_FACTOR),
            "Skew": series.skew(),
            "Corr r_m": series.corr(subset["r_m"]),
        }
        if include_value_corr:
            row["Corr r_v"] = series.corr(subset["r_v"])
        rows.append(row)
    return pd.DataFrame(rows).set_index("Series").sort_index()

summary_columns = [
    "r_mom_ff",
    "r_mom",
    "r_mom1",
    "r_mom10",
    "r_momSU",
    "r_momBD",
    "r_momU",
    "r_momU_ff",
    "r_momD1",
    "r_momD3",
    "r_momD5",
    "r_momS",
    "r_momB",
]
window_summary_cache = build_window_summary(summary_columns, sample_windows)

table1 = window_summary_cache.loc["r_mom_ff", metric_order].rename(columns=metric_rename).round(2)
print("Table 1. Annualized summary for r_mom:FF (excess returns by subsample).")
display(table1)

component_series = ["r_mom", "r_mom1", "r_mom10", "r_momSU", "r_momBD"]
component_summary = (
    window_summary_cache
    .loc[component_series, metric_order]
    .rename(index=lambda idx: series_labels.get(idx, idx), level="Series")
    .rename(columns=metric_rename)
)
component_view = component_summary.swaplevel("Series", "Period").sort_index().unstack("Series").round(2)
print("Supporting stats for the legs of the long-short trade (annualized %, monthly data).")
display(component_view)



Table 1 shows that `r_mom:FF` earned 7.4% per year with a 0.45 Sharpe ratio over 1927?2024, but performance flip-flopped across subsamples: 8.8% (Sharpe 0.55) before 1994, 10.3% (Sharpe 0.59) from 1994?2008, and ?1.3% (Sharpe ?0.08) since 2009. The risk profile stayed near 16% annualized volatility, so the return collapse?not higher volatility?drove the post-2009 drawdown.

The supporting table clarifies why: the short leg (losers) delivered ?7.3% annually in 1994?2008 but +16.1% since 2009, while big losers (`r_momBD`) swung from ?1.2% to +16.0%. Winners (`r_mom(10)`/`r_momSU`) remained strong in every window, so recent underperformance is mostly a short-leg squeeze during sharp factor reversals. That history answers (b): momentum's mean payoff is highly regime-dependent even though its negative correlation to the market (about ?0.3) and value factor (about ?0.4) stayed intact.

Regarding (c), the data still support AQR's diversification argument. Even if expected returns net of costs were near zero, the consistently negative correlation with the market and value factors plus the convex payoff to crisis periods (pre-2009) make momentum a valuable completion strategy?provided investors tolerate the possibility of decade-long soft patches like 2009?2024.



# 2.2
Whether a long-only implementation of momentum is valuable. (a) Report the requested statistics for `r_mom:FF` and the long-only `r_momU:FF` over 1994?2024. (b) Compare mean, volatility, and Sharpe ratios. (c) Compare diversification relative to the market and value factors. (d) Plot the cumulative products of (1 + r_mom:FF) and (1 + r_momU:FF) for 1994?2024.


In [ ]:

analysis_start, analysis_end = "1994-01-31", "2024-12-31"
table2_results = compute_period_stats(["r_mom_ff", "r_momU_ff", "r_momU"], analysis_start, analysis_end)
table2_display = (
    table2_results
    .loc[["r_mom_ff", "r_momU_ff"], metric_order]
    .rename(index=lambda idx: series_labels.get(idx, idx))
    .rename(columns=metric_rename)
    .round(2)
)
print("Table 2. Long-short vs long-only momentum (1994?2024, annualized %).")
display(table2_display)

plot_window = monthly.loc[(monthly.index >= analysis_start) & (monthly.index <= analysis_end), ["r_mom_ff", "r_momU_ff"]]
cum_growth = (1 + plot_window).cumprod()
ax = cum_growth.plot(figsize=(10, 4))
ax.axhline(1.0, color="gray", linewidth=0.8, linestyle="--")
ax.set_ylabel("Growth of $1")
ax.set_title("Cumulative wealth from momentum implementations (1994?2024)")
ax.legend(["r_mom:FF", "Long-only (r_momU:FF)"])
plt.show()



(a) Table 2 shows that long-short momentum earned 4.3% per year with 16.7% volatility (Sharpe 0.26), while the long-only `r_momU:FF` delivered 11.6% with 17.8% volatility (Sharpe 0.65). The decile-based long-only variant `r_momU` is similar (10.2% mean, Sharpe 0.64), confirming the FF-style construction.

(b) Long-only delivers the higher mean and Sharpe, but only by accepting that most of the payoff is simply equity beta: its volatility is slightly higher and its return path mirrors the market.

(c) Diversification differs sharply. `r_mom:FF` keeps a ?0.31 correlation to the market and ?0.21 to value, retaining the classic defensive, contrarian profile. `r_momU:FF` ties itself to the market with a +0.90 correlation (and only ?0.12 to value), so it would not help an investor seeking a hedge when equities fall.

(d) The cumulative wealth plot highlights the trade-off: $1 invested in long-only momentum grew to roughly $22 by 2024 versus only $2.4 for `r_mom:FF`, but the former crashes whenever equities sell off, whereas the latter treads water because its short leg offsets market directionality. The choice depends on whether one prioritizes diversification or raw compounded returns.



# 2.3
Is momentum just data mining, or is it robust to the winner/loser thresholds? (a) Compute Table 3 for the one-, three-, and five-decile long-short portfolios over 1994?2024. (b) Relate the trade-offs to the lecture discussion. (c) Advise whether AQR should consider 1-decile or 5-decile cuts for the retail product. (d) Compare the 3-decile construction to `r_mom:FF` from Table 2.


In [ ]:

analysis_start, analysis_end = "1994-01-31", "2024-12-31"
table3_results = compute_period_stats(["r_momD1", "r_momD3", "r_momD5"], analysis_start, analysis_end)
table3_display = (
    table3_results
    .loc[:, metric_order]
    .rename(index=lambda idx: series_labels.get(idx, idx))
    .rename(columns=metric_rename)
    .round(2)
)
print("Table 3. Robustness of long-short constructions (1994?2024, annualized %).")
display(table3_display)



(a) The table confirms the expected pattern: concentrating in the top and bottom deciles (`1-decile long-short`) yields the highest mean (8.1%) but nearly 30% volatility and a Sharpe of only 0.27 because of severe negative skew (?1.30). Spreading to 3 or 5 deciles lowers volatility to 19% and 13% respectively, yet the mean falls to 3.0% and 1.7%, so Sharpe ratios compress to 0.15?0.13.

(b) This mirrors theory: tighter winner/loser buckets maximize signal strength but create fragile portfolios that are more exposed to crowding and liquidity spirals, hence the fat left tails. Broader buckets dilute alpha yet deliver smoother risk.

(c) AQR's retail mandate probably should not rely purely on the 1-decile version?its drawdown risk is high and would be hard to explain to non-institutional clients. Likewise, the 5-decile version seems too diluted to matter. A 3-decile (or FF-style size-segmented) compromise balances tradability with a still-meaningful payoff.

(d) Comparing the 3-decile construction (mean 3.0%, Sharpe 0.15) to `r_mom:FF` (mean 4.3%, Sharpe 0.26) highlights the benefit of the FF refinements: splitting by size and equal-weighting small and big legs adds about 1.3 percentage points of annual return and lowers correlation to value, so the FF recipe remains the stronger benchmark.



# 2.4
Does implementing momentum require trading many small stocks, thereby inflating costs? (a) Fill Table 4 for 1994?2024 using `r_mom:FF`, the small-stock momentum spread (`r_momS`), and the big-stock spread (`r_momB`). (b) Discuss whether small stocks drive the strategy and whether a large-cap implementation still works.


In [ ]:

analysis_start, analysis_end = "1994-01-31", "2024-12-31"
table4_results = compute_period_stats(["r_mom_ff", "r_momS", "r_momB"], analysis_start, analysis_end, include_value_corr=False)
table4_columns = ["Mean % (ann)", "Vol % (ann)", "Sharpe", "Skew", "Corr r_m"]
table4_display = (
    table4_results
    .loc[:, table4_columns]
    .rename(index=lambda idx: series_labels.get(idx, idx))
    .rename(columns={
        "Mean % (ann)": "Mean (% p.a.)",
        "Vol % (ann)": "Vol (% p.a.)",
        "Corr r_m": "Corr vs r_m",
    })
    .round(2)
)
print("Table 4. Momentum in small vs big stocks (1994?2024, annualized %).")
display(table4_display)



(a) Small-stock momentum delivered 6.1% per year with 16.9% volatility and a Sharpe of 0.36, versus big-stock momentum at 2.5% with 18.2% volatility and a Sharpe of 0.14. The combined `r_mom:FF` (equal small/big) sits in between at 4.3% and a Sharpe of 0.26. All three retain mildly negative correlations to the market (about ?0.31 for small, ?0.28 for big).

(b) Small caps clearly contribute most of the premium, but large caps still provide a positive (if modest) excess return with similar diversification. For a retail product worried about trading costs, emphasizing larger, more liquid names is feasible, albeit with lower expected alpha; blending small and big baskets, or scaling exposure by turnover budgets, seems prudent.



# 2.5
Conclusion: assess the proposed AQR retail momentum product. Does it capture the essential features of the Fama?French construction, and what (if any) modifications are warranted?


In [ ]:

conclusion_sources = {
    "r_mom_ff": (table2_results, "Long-short r_mom:FF"),
    "r_momU_ff": (table2_results, "Long-only r_momU:FF"),
    "r_momD3": (table3_results, "3-decile long-short"),
    "r_momS": (table4_results, "Small-stock momentum"),
    "r_momB": (table4_results, "Big-stock momentum"),
}
conclusion_rows = []
for key, (source, label) in conclusion_sources.items():
    conclusion_rows.append({
        "Series": label,
        "Mean (% p.a.)": source.loc[key, "Mean % (ann)"],
        "Sharpe": source.loc[key, "Sharpe"],
    })
conclusion_table = pd.DataFrame(conclusion_rows).set_index("Series").round(2)
print("Key statistics referenced in the conclusion (1994?2024).")
display(conclusion_table)



The retail proposal should acknowledge that the traditional long-short momentum sleeve (`r_mom:FF`) has been flat to slightly negative since 2009 (?1.3% mean, Sharpe ?0.08), yet it still supplies valuable diversification because of its ?0.3 correlation to both the market and the value factor. A pure long-only implementation, by contrast, now looks like an equity-tilted growth fund: `r_momU:FF` boasts an 11.6% mean and 0.65 Sharpe but carries 0.90 market correlation, so clients expecting a diversifier would be disappointed when equities sell off.

To remain faithful to Fama?French, the retail product should keep the size-segmented 3-decile construction (or hedge a portion of the market beta via index futures) so that the strategy's return is not dominated by the market rally effect seen in the long-only wealth chart. The analysis also suggests two concrete adjustments:

1. **Risk-managed exposure.** Scaling the position when the short leg (losers) is rallying?as in 2009?2024?would have limited the drawdown that cratered recent performance. A simple volatility or drawdown-based throttle could improve realized Sharpe without abandoning the FF recipe.
2. **Blend small and large baskets with turnover controls.** Small-stock momentum (Sharpe 0.36) still provides most of the alpha, but big-stock momentum remains positive (Sharpe 0.14) and is cheaper to trade. AQR can overweight larger names in the retail fund while keeping a separate, capacity-constrained sleeve for small names, thereby retaining some of the FF diversification while meeting liquidity constraints.

Netting these points, my recommendation is to market the product as a diversified momentum sleeve only if some short exposure (or overlay hedge) stays in place; otherwise, position it as a tactical, equity-sensitive satellite strategy and pair it with other diversifiers inside the client portfolio.
